In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def get_sp500():
    """
    Get list of S&P 500 stocks from wikipedia
    """
    sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    sp500_tickers = sp500_tickers[0]

    tickers = sp500_tickers['Symbol'].values.tolist()
    return tickers

companies = get_sp500()

In [3]:
import os
from pandas_datareader import data as pdr
import pandas as pd
#import fix_yahoo_finance as yf
import yfinance as yf

yf.pdr_override()

START_DATE = "2021-01-01"
END_DATE = "2021-04-01"

def build_stock_dataset(start=START_DATE, end=END_DATE):
    """
    Creates the dataset containing all stock prices
    :returns: stock_prices.csv
    """
    ticker_list = get_sp500()

    # Get all Adjusted Close prices for all the tickers in our list,
    # between START_DATE and END_DATE
    all_data = pdr.get_data_yahoo(ticker_list, start, end)
    stock_data = all_data["Adj Close"]

    # Remove any columns that hold no data, and print their tickers.
    stock_data.dropna(how="all", axis=1, inplace=True)
    missing_tickers = [
        ticker for ticker in ticker_list if ticker.upper() not in stock_data.columns
    ]
    print(f"{len(missing_tickers)} tickers are missing: \n {missing_tickers} ")
    # If there are only some missing datapoints, forward fill.
    stock_data.ffill(inplace=True)
    stock_data.to_csv("stock_prices.csv")
    
build_stock_dataset()

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
2 tickers are missing: 
 ['BRK.B', 'BF.B'] 


/home/h4x3d/.pyenv/versions/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/h4x3d/.pyenv/versions/3.6.6/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [4]:
import sys
sys.path.append("../")
import edhec_risk_kit as erk

In [5]:
stocks = pd.read_csv("stock_prices.csv")
stocks.head()

,Date,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2021-01-04,118.459747,15.13,157.123825,128.997803,102.939789,95.746490,316.730011,108.274658,254.798035,...,65.173874,142.429993,40.221066,53.660934,99.407997,105.315231,152.927170,378.130005,42.397789,163.093948
1,2021-01-05,119.428276,15.43,156.954056,130.592697,104.004242,96.996651,322.600006,109.614319,256.248596,...,64.538605,144.229996,42.159431,55.198383,99.288330,105.325180,155.573135,380.570007,43.069359,164.230499
2,2021-01-06,122.703293,15.52,166.021576,126.196747,103.105804,105.340981,321.609985,109.386086,259.050293,...,65.967941,141.220001,43.235222,57.863960,103.476448,105.902420,160.006393,394.820007,47.908611,166.643158
3,2021-01-07,125.968323,15.38,167.439621,130.502991,104.209320,109.270058,323.559998,110.447891,261.494385,...,64.677574,149.710007,43.574436,57.564457,108.173134,105.096275,159.057846,409.100006,49.370266,166.493607
4,2021-01-08,126.866951,15.13,169.826340,131.629379,104.756195,109.170837,324.059998,110.755516,262.448151,...,65.273132,147.990005,44.059025,58.083595,107.076241,106.638885,158.708374,405.470001,48.116013,167.600235


In [6]:
stocks = stocks[stocks.columns[1:]]/stocks[stocks.columns[1:]].shift(1) - 1
stocks.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.008176,0.019828,-0.001080,0.012364,0.010341,0.013057,0.018533,0.012373,0.005693,0.000721,...,-0.009747,0.012638,0.048193,0.028651,-0.001204,0.000094,0.017302,0.006453,0.015840,0.006969
2,0.027422,0.005833,0.057772,-0.033662,-0.008638,0.086027,-0.003069,-0.002082,0.010934,-0.039902,...,0.022147,-0.020869,0.025517,0.048291,0.042181,0.005481,0.028496,0.037444,0.112360,0.014691
3,0.026609,-0.009021,0.008541,0.034123,0.010703,0.037299,0.006063,0.009707,0.009435,0.024512,...,-0.019561,0.060119,0.007846,-0.005176,0.045389,-0.007612,-0.005928,0.036168,0.030509,-0.000897
4,0.007134,-0.016255,0.014254,0.008631,0.005248,-0.000908,0.001545,0.002785,0.003647,0.015406,...,0.009208,-0.011489,0.011121,0.009018,-0.010140,0.014678,-0.002197,-0.008873,-0.025405,0.006647


In [11]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (2000,500)
(stocks*100).plot.bar() 

<AxesSubplot:>

ValueError: Image size of 144000x36000 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 144000x36000 with 1 Axes>